In [1]:
import numpy as np
import tensorflow as tf

/Users/yunsungsong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sample = ' my name is peter'
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

In [4]:
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sample)-sequence_length):
    x_str = sample[i:i+sequence_length]
    y_str = sample[i+1:i+sequence_length+1]
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, shape=[None,sequence_length])
Y = tf.placeholder(tf.int32, shape=[None,sequence_length])
x_one_hot = tf.one_hot(X, input_dim)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state,
                                    dtype=tf.float32)
X_for_fc = tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size,sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                                weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)


[7, 5, 2, 7, 10, 8, 5, 9, 7, 4, 3, 7, 0, 9, 6, 9, 1]
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


ValueError: Shape (16, ?) must have rank at least 3

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    l,_ = sess.run([loss,train], feed_dict={X:dataX, Y:dataY})
    if step % 100 == 0:
#         result = sess.run(prediction, feed_dict={X:x_data})
#         result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("step:",step,"Loss:",l)